# Simple Data Generation for Tableau

We will store the data in data frames to aid with exporting to \*.csv files

In [3]:
using DataFrames

## Time to First Event Data

We initialize the data frame with 3 columns and 1024 samples

1. The first column contains the subject `subject`, randomly selected from 64 identifiers.
2. The second column contains the precursor to `time`, the cumulative sum of random increments from 1 to 8
3. The third column contains the `event`, randomly selected from a central distribution on the letters A to E

In [14]:
longitudinal = DataFrame(
    subject = sort(rand(1:64, 1024)),
    time = cumsum(rand(1:8, 1024)),
    event = rand(['A', 'B', 'B', 'C', 'C', 'C', 'C', 'D', 'D', 'E'], 1024)
);
longitudinal[1:8, :]

,subject,time,event
1,1,5,C
2,1,7,B
3,1,15,D
4,1,20,B
5,1,24,C
6,1,32,C
7,1,40,C
8,1,45,C


Notice that the time is increasing across all subjects. Instead we want to be the cumulative sum strictly within each subject. To accomplish this we use logical indexing to pull the last time from each subject and subtract it from the times of the next subject.

First find the logic index of the row before the subject changed.

In [15]:
longitudinal[:before] = [longitudinal[:subject][1:end-1] .!= longitudinal[:subject][2:end];false];
longitudinal[1:8, :]

,subject,time,event,before
1,1,5,C,false
2,1,7,B,false
3,1,15,D,false
4,1,20,B,false
5,1,24,C,false
6,1,32,C,false
7,1,40,C,false
8,1,45,C,false


Similarly find the logical index of the row after the subject changed.

In [16]:
longitudinal[:after] = [false; longitudinal[:before][1:end-1]];
longitudinal[1:8, :]

,subject,time,event,before,after
1,1,5,C,false,false
2,1,7,B,false,false
3,1,15,D,false,false
4,1,20,B,false,false
5,1,24,C,false,false
6,1,32,C,false,false
7,1,40,C,false,false
8,1,45,C,false,false


Next initialize a column to contain the shift of baseline time for each subject.

In [17]:
longitudinal[:shift] = zeros(Int, 1024);
longitudinal[1:8, :]

,subject,time,event,before,after,shift
1,1,5,C,false,false,0
2,1,7,B,false,false,0
3,1,15,D,false,false,0
4,1,20,B,false,false,0
5,1,24,C,false,false,0
6,1,32,C,false,false,0
7,1,40,C,false,false,0
8,1,45,C,false,false,0


Into the baseline shift of each row following the change in subject place the difference between the successive start times of each subject.

In [18]:
longitudinal[:shift][longitudinal[:after]] = 
    longitudinal[:time][longitudinal[:before]] - 
    [0; longitudinal[:time][longitudinal[:before]][1:end-1]];
longitudinal[1:8, :]

,subject,time,event,before,after,shift
1,1,5,C,false,false,0
2,1,7,B,false,false,0
3,1,15,D,false,false,0
4,1,20,B,false,false,0
5,1,24,C,false,false,0
6,1,32,C,false,false,0
7,1,40,C,false,false,0
8,1,45,C,false,false,0


Fill in the zeroes by running a cummulative sum over the shift column.

In [19]:
longitudinal[:shift] = cumsum(longitudinal[:shift]);
longitudinal[1:8, :]

,subject,time,event,before,after,shift
1,1,5,C,false,false,0
2,1,7,B,false,false,0
3,1,15,D,false,false,0
4,1,20,B,false,false,0
5,1,24,C,false,false,0
6,1,32,C,false,false,0
7,1,40,C,false,false,0
8,1,45,C,false,false,0


Rectify the time for each subject by subtracting the baseline shift.

In [20]:
longitudinal[:time] = longitudinal[:time] - longitudinal[:shift];
longitudinal[1:8, :]

,subject,time,event,before,after,shift
1,1,5,C,false,false,0
2,1,7,B,false,false,0
3,1,15,D,false,false,0
4,1,20,B,false,false,0
5,1,24,C,false,false,0
6,1,32,C,false,false,0
7,1,40,C,false,false,0
8,1,45,C,false,false,0


We can now export the data, choosing the first 3 columns.

In [21]:
writetable("longitudinal.csv", longitudinal[[:subject, :time, :event]])

## Contingency Data

The goal is to generate four columns of dimensional classifiers, as two pairs of dependent dimensions. We start by initializing the two distributions to sample:

In [52]:
distributionone = [
    ["A" -1],
    ["A" -1],
    ["A" -1],
    ["A" 0],
    ["A" 0],
    ["A" 1],
    ["B" -1],
    ["B" 0],
    ["B" 0],
    ["B" 1],
    ["B" 1],
    ["B" 1]
];
distributiontwo = [
    ["C" -2],
    ["C" -1],
    ["C" -1],
    ["C" 0],
    ["C" 0],
    ["C" 0],
    ["C" 1],
    ["C" 1],
    ["C" 1],
    ["C" 1],
    ["C" 2],
    ["C" 2],
    ["C" 2],
    ["C" 2],
    ["C" 2],
    ["D" -2],
    ["D" -1],
    ["D" -1],
    ["D" 0],
    ["D" 0],
    ["D" 0],
    ["D" 1],
    ["D" 1],
    ["D" 2],
    ["E" -2],
    ["E" -2],
    ["E" -2],
    ["E" -1],
    ["E" -1],
    ["E" 0],
    ["E" 1],
    ["E" 1],
    ["E" 2],
    ["E" 2],
    ["E" 2],
    ["F" 2],
    ["F" 1],
    ["F" 1],
    ["F" 0],
    ["F" 0],
    ["F" 0],
    ["F" -1],
    ["F" -1],
    ["F" -1],
    ["F" -1],
    ["F" -2],
    ["F" -2],
    ["F" -2],
    ["F" -2],
    ["F" -2]
];

We then generate another 1024 samples to populate a data frame.

In [53]:
contingency = DataFrame(
    sample = 1:1024,
    preonetwo = rand(distributionone, 1024),
    prethreefour = rand(distributiontwo, 1024)
);
contingency[1:8, :]

,sample,preonetwo,prethreefour
1,1,"Any[""B"" 0]","Any[""C"" 0]"
2,2,"Any[""B"" 0]","Any[""C"" 0]"
3,3,"Any[""B"" 1]","Any[""C"" -1]"
4,4,"Any[""B"" -1]","Any[""D"" -1]"
5,5,"Any[""B"" 1]","Any[""F"" 0]"
6,6,"Any[""B"" 0]","Any[""F"" -2]"
7,7,"Any[""B"" 1]","Any[""C"" -1]"
8,8,"Any[""B"" -1]","Any[""C"" 1]"


Next we extract the individual column elements into dimension columns.

In [54]:
contingency[:dimensionone] = vcat(contingency[:, :preonetwo]...)[:, 1];
contingency[:dimensiontwo] = vcat(contingency[:, :preonetwo]...)[:, 2];
contingency[:dimensionthree] = vcat(contingency[:, :prethreefour]...)[:, 1];
contingency[:dimensionfour] = vcat(contingency[:, :prethreefour]...)[:, 2];
contingency[1:8, :]

,sample,preonetwo,prethreefour,dimensionone,dimensiontwo,dimensionthree,dimensionfour
1,1,"Any[""B"" 0]","Any[""C"" 0]",B,0,C,0
2,2,"Any[""B"" 0]","Any[""C"" 0]",B,0,C,0
3,3,"Any[""B"" 1]","Any[""C"" -1]",B,1,C,-1
4,4,"Any[""B"" -1]","Any[""D"" -1]",B,-1,D,-1
5,5,"Any[""B"" 1]","Any[""F"" 0]",B,1,F,0
6,6,"Any[""B"" 0]","Any[""F"" -2]",B,0,F,-2
7,7,"Any[""B"" 1]","Any[""C"" -1]",B,1,C,-1
8,8,"Any[""B"" -1]","Any[""C"" 1]",B,-1,C,1


Finally we export our contingency table data.

In [55]:
writetable("contingency.csv", contingency[[:sample, :dimensionone, :dimensiontwo, :dimensionthree, :dimensionfour]])